# Team Analysis
==========================================================================================

### Thinking
* Players on better teams will play with better players and therefore will perform better
    * Opponents will not be able to focus in on one specific player
* Playing on a team with good defense will only for more offensive snaps
* A good team plays to its strengths: good players will get the ball more
    * Having a smart offensive coordinator or head coach that has a playbook that involves their best players is ideal
* However, a good team may simply just have good players
    * This means that sometimes, your player will perform well regardless of team: they are simply effective 
    * In addition, there is only one ball: stats cannot be shared
        * Playing on a poor team would allow one great player to take in all the stats
    * Does playing on a good team improve or decrease your fantasy production?
    * In reality, this would mainly apply to teams with multiple superstars
   
==========================================================================================
### Data Collection
* Data is available from 2016-2021 (2022, but I don't want to include that data)
* Players have their stats available in this same time period, but advanced stats and defensive aren't tracked until 2018
    * Therefore, I will exclude looking at advanced stats in general in the first bit
    
==========================================================================================
### Method of Analysis
##### Numbers:
* Will calculate the difference of a player's draft position and final standing and its 
    * For example: a player with an ADP (Average Draft Position) of RB12 but finishes the year as the RB5 would be +7
        * Flipping the ADP and Finish would be a -7 rating
        * However, finishing at RB5 from ADP12 is more important than finishing RB24 from ADP31
            * The gap in points is simply too large
            
##### Cleaning + Preparing
* Bring in the data and store in pandas dataframes
    * Need to bring in:
        * ADP (2016-2021Just Overall)
        * Final Seasonal Numbers
        * RedZone Passing, Receiving, Rushing
        * TeamOffense:
            * Conversions
            * Drive averages
            * Passing Offense
            * Rushing Offense
            * Scoring Offense
* Create a Hashing System to link up team names with their abreviations
* Link up player data (Player Team -> Team scoring/passing/receiving/rushing)
* Divide up players by position for comparison
* Scale the numbers (for SVM)

##### Clustering
* Graph a couple of seemingly import attributes
* See if KMeans or MeanShift can cluster the data into groups, may take a long time

##### Modeling
* Will use a SVM and DecisionTree (Maybe RandomForest) to see if there is any trends
* Test on 2022 Seasonal Data while training on these 5-6 years

In [ ]:
### Selecting data to load into list of pandas dataframes
import numpy as np
import pandas as pd
adp = [] #ADP
rnk = [] #End of Season FantasyRankings
rzp = [] #RedZone Passing
rzru = [] #RedZone Rushing
rzre = [] #RedZone Receiving
convs = [] #Conversions
das = [] #driveAverages
pO = [] #passingOffense
rO = [] #rushingOffense
sO = [] #scoringOffense
tO = [] #teamOffense
for i in range(2016, 2022):
    address = "Data/ADP/FantasyPros_" + str(i) + "_Overall_ADP_Rankings.csv"
    temp = pd.read_csv(address, index_col=[1])
    temp.fillna(0, inplace=True)
    adp.append(temp)
    address = "Data/FantasyRankings/" + str(i) + ".csv"
    temp = pd.read_csv(address, index_col=[1])
    temp.fillna(0, inplace=True)
    rnk.append(temp)
    address = "Data/PlayerStats/Passing/redZonePassing/" + str(i) + ".csv"
    temp = pd.read_csv(address, index_col=[0])
    temp.fillna(0, inplace=True)
    rzp.append(temp)
    address = "Data/PlayerStats/Rushing/redZoneRushing/" + str(i) + ".csv"
    temp = pd.read_csv(address, index_col=[0])
    temp.fillna(0, inplace=True)
    rzru.append(temp)
    address = "Data/PlayerStats/Receiving/redZoneReceiving/" + str(i) + ".csv"
    temp = pd.read_csv(address, index_col=[0])
    temp.fillna(0, inplace=True)
    rzre.append(temp)
    address = "Data/TeamOffense/conversions/" + str(i) + ".csv"
    temp = pd.read_csv(address, index_col=[1])
    temp.fillna(0, inplace=True)
    convs.append(temp)
    address = "Data/TeamOffense/driveAverages/" + str(i) + ".csv"
    temp = pd.read_csv(address, index_col=[1])
    temp.fillna(0, inplace=True)
    das.append(temp)
    address = "Data/TeamOffense/passingOffense/" + str(i) + ".csv"
    temp = pd.read_csv(address, index_col=[1])
    temp.fillna(0, inplace=True)
    pO.append(temp)
    address = "Data/TeamOffense/rushingOffense/" + str(i) + ".csv"
    temp = pd.read_csv(address, index_col=[1])
    temp.fillna(0, inplace=True)
    rO.append(temp)
    address = "Data/TeamOffense/scoringOffense/" + str(i) + ".csv"
    temp = pd.read_csv(address, index_col=[1])
    temp.fillna(0, inplace=True)
    sO.append(temp)
    address = "Data/TeamOffense/teamOffense/" + str(i) + ".csv"
    temp = pd.read_csv(address, index_col=[1])
    temp.fillna(0, inplace=True)
    tO.append(temp)
teamdata = []
for i in range(0, 6):
    df = pd.merge(convs[i], das[i], left_index=True, right_index=True)
    df = pd.merge(df, pO[i], left_index=True, right_index=True)
    df = pd.merge(df, rO[i], left_index=True, right_index=True)
    df = pd.merge(df, sO[i], left_index=True, right_index=True)
    df = pd.merge(df, tO[i], left_index=True, right_index=True)
    teamdata.append(df)
for i in range(5):
    address = "Data/Merged/" + str(i+ 2017) + ".csv"
    teamdata[i].to_csv(address)
for i in range(6):
    adp[i] = adp[i][['Player', 'POS', 'AVG']]
    rnk[i].set_index('Player')
    rnk[i] = rnk[i].drop(['Unnamed: 0'], axis=1, inplace=True)
for i in range(6):
    rnk[i]['Player'] = rnk[i]['Player'].str.replace('*', '')
    rnk[i]['Player'] = rnk[i]['Player'].str.replace('+', '')
for i in range(6):
    t1 = adp[i].copy(deep=True)
    t2 = rnk[i].copy(deep=True)
    t3 = rzp[i].copy(deep=True)
    t4 = rzru[i].copy(deep=True)
    t5 = rzre[i].copy(deep=True)

    t1 = t1.join(t2, on='Player')
    t1 = t1.join(t3, on='Player', lsuffix='rank', rsuffix='rzpa')
    t1 = t1.join(t4, on='Player', lsuffix='rzpa', rsuffix='rzru')
    t1 = t1.join(t5, on='Player', lsuffix='rzru', rsuffix='rzre')
    t1.fillna(0, inplace=True)
    address = "Data/MergedPlayers/" + str(i+2016) + ".csv"
    t1.to_csv(address)
for i in range(1,6):
    rnk[i].drop(['Linkrzru', '-9999rzru', 'Tmrzre', '-9999rank', 'Tmrzpa', '-9999rzre', 'Link', 'Linkrzpa', '-9999rzpa', 'Linkrzpa', 'Tmrzru', '-9999rank'], axis=1, inplace=True)
    rnk[i]['POS'] = rnk[i]['POS'].astype('string')
    rnk[i]['Tmrank'] = rnk[i]['Tmrank'].astype('string')
    rnk[i]['FantPos'] = rnk[i]['FantPos'].astype('string')
    rnk[i]['%Tgt'] = rnk[i]['%Tgt'].astype("string")
    rnk[i]['%Tgt'] = rnk[i]['%Tgt'].str.replace('%', '')
    rnk[i]['%Tgt'] = rnk[i]['%Tgt'].astype(float)
    rnk[i]['%Tgt.1'] = rnk[i]['%Tgt.1'].astype("string")
    rnk[i]['%Tgt.1'] = rnk[i]['%Tgt.1'].str.replace('%', '')
    rnk[i]['%Tgt.1'] = rnk[i]['%Tgt.1'].astype(float)
    
    rnk[i]['Ctch%'] = rnk[i]['Ctch%'].astype("string")
    rnk[i]['Ctch%'] = rnk[i]['Ctch%'].str.replace('%', '')
    rnk[i]['Ctch%'] = rnk[i]['Ctch%'].astype(float)
    rnk[i]['Ctch%.1'] = rnk[i]['Ctch%.1'].astype("string")
    rnk[i]['Ctch%.1'] = rnk[i]['Ctch%.1'].str.replace('%', '')
    rnk[i]['Ctch%.1'] = rnk[i]['Ctch%.1'].astype(float)
    rnk[i]['%Rush.2'] = rnk[i]['%Rush.2'].astype("string")
    rnk[i]['%Rush.2'] = rnk[i]['%Rush.2'].str.replace('%', '')
    rnk[i]['%Rush.2'] = rnk[i]['%Rush.2'].astype(float)
    rnk[i]['%Rush.1'] = rnk[i]['%Rush.1'].astype("string")
    rnk[i]['%Rush.1'] = rnk[i]['%Rush.1'].str.replace('%', '')
    rnk[i]['%Rush.1'] = rnk[i]['%Rush.1'].astype(float)
    rnk[i]['%Rush'] = rnk[i]['%Rush'].astype("string")
    rnk[i]['%Rush'] = rnk[i]['%Rush'].str.replace('%', '')
    rnk[i]['%Rush'] = rnk[i]['%Rush'].astype(float)
for i in range(6):
    team[i]['3D%'] = team[i]['3D%'].astype("string")
    team[i]['3D%'] = team[i]['3D%'].str.replace('%', '')
    team[i]['3D%'] = team[i]['3D%'].astype(float)
    
    team[i]['4D%'] = team[i]['4D%'].astype("string")
    team[i]['4D%'] = team[i]['4D%'].str.replace('%', '')
    team[i]['4D%'] = team[i]['4D%'].astype(float)
    
    team[i]['RZPct'] = team[i]['RZPct'].astype("string")
    team[i]['RZPct'] = team[i]['RZPct'].str.replace('%', '')
    team[i]['RZPct'] = team[i]['RZPct'].astype(float)
    
    team[i]['Start'] = team[i]['Start'].astype("string")
    team[i]['Start'] = team[i]['Start'].str.replace('Own ', '')
    team[i]['Start'] = team[i]['Start'].astype(float)
    
    team[i]['Time'] = team[i]['Time'].astype('string')
    team[i]['Time'] = team[i]['Time'].str.replace(':','.')
    team[i]['Time'] = team[i]['Time'].astype(float)
    
    address = "Data/Merged/" + str(i+2016) + ".csv"
    team[i].to_csv(address)

In [63]:
# Creating a set of the teams
# Due to moving of team and team name changes, this is not as simple as it would seem
namesHash = {"DET": "Detroit Lions",
"GNB": "Green Bay Packers",
"MIN": "Minnesota Vikings",
"JAX": "Jacksonville Jaguars",
"LVR": "Las Vegas Raiders",
"PHI": "Philadelphia Eagles",
"NOR": "New Orleans Saints",
"SFO": "San Francisco 49ers",
"HOU": "Houston Texans",
"MIA": "Miami Dolphins",
"PIT": "Pittsburgh Steelers",
"SEA": "Seattle Seahawks",
"CHI": "Chicago Bears",
"IND": "Indianapolis Colts",
"TEN": "Tennessee Titans",
"NYJ": "New York Jets",
"LAR": "Los Angeles Rams",
"DAL": "Dallas Cowboys",
"2TM": "",
"KAN": "Kansas City Chiefs",
"SDG": "San Diego Chargers",
"3TM": "",
"LAC": "Los Angeles Chargers",
"BUF": "Buffalo Bills",
"BAL": "Baltimore Ravens",
"ATL": "Atlanta Falcons",
"OAK": "Oakland Raiders",
"DEN": "Denver Broncos",
"CLE": "Cleveland Browns",
"NWE": "New England Patriots",
"TAM": "Tampa Bay Buccaneers",
"NYG": "New York Giants",
"CIN": "Cincinnati Bengals",
"ARI": "Arizona Cardinals",
"WAS": "Washington Redskins",
"CAR": "Carolina Panthers",}    
namesHash1 = {"DET": "Detroit Lions",
"GNB": "Green Bay Packers",
"MIN": "Minnesota Vikings",
"JAX": "Jacksonville Jaguars",
"LVR": "Las Vegas Raiders",
"PHI": "Philadelphia Eagles",
"NOR": "New Orleans Saints",
"SFO": "San Francisco 49ers",
"HOU": "Houston Texans",
"MIA": "Miami Dolphins",
"PIT": "Pittsburgh Steelers",
"SEA": "Seattle Seahawks",
"CHI": "Chicago Bears",
"IND": "Indianapolis Colts",
"TEN": "Tennessee Titans",
"NYJ": "New York Jets",
"LAR": "Los Angeles Rams",
"DAL": "Dallas Cowboys",
"2TM": "",
"KAN": "Kansas City Chiefs",
"SDG": "San Diego Chargers",
"3TM": "",
"LAC": "Los Angeles Chargers",
"BUF": "Buffalo Bills",
"BAL": "Baltimore Ravens",
"ATL": "Atlanta Falcons",
"OAK": "Oakland Raiders",
"DEN": "Denver Broncos",
"CLE": "Cleveland Browns",
"NWE": "New England Patriots",
"TAM": "Tampa Bay Buccaneers",
"NYG": "New York Giants",
"CIN": "Cincinnati Bengals",
"ARI": "Arizona Cardinals",
"WAS": "Washington Football Team",
"CAR": "Carolina Panthers",} 

In [104]:
import numpy as np
import pandas as pd

rnk = []
team = []

for i in range(2016, 2022):
    address = "Data/Merged/" + str(i) + ".csv"
    temp = pd.read_csv(address, index_col=[0])
    temp.fillna(0, inplace=True)
    team.append(temp)
    address = "Data/MergedPlayers/" + str(i) + ".csv"
    temp = pd.read_csv(address, index_col=[0])
    temp.fillna(0, inplace=True)
    temp['POS'] = temp['POS'].astype("string")
    temp['Tmrank'] = temp['Tmrank'].astype("string")
    temp['FantPos'] = temp['FantPos'].astype("string")
    temp = temp[temp['Tmrank'] !="0"]
    rnk.append(temp)

In [102]:
rnk[0].info()

<class 'pandas.core.frame.DataFrame'>
Index: 334 entries, Antonio Brown to Ed Dickson
Data columns (total 72 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Rank        334 non-null    float64
 1   POS         334 non-null    object 
 2   AVG         334 non-null    float64
 3   Rk          334 non-null    float64
 4   Tmrank      334 non-null    object 
 5   FantPos     334 non-null    object 
 6   Age         334 non-null    float64
 7   G           334 non-null    float64
 8   GS          334 non-null    float64
 9   Cmprank     334 non-null    float64
 10  Attrank     334 non-null    float64
 11  Ydsrank     334 non-null    float64
 12  TDrank      334 non-null    float64
 13  Intrank     334 non-null    float64
 14  Att.1rank   334 non-null    float64
 15  Yds.1rank   334 non-null    float64
 16  Y/A         334 non-null    float64
 17  TD.1rank    334 non-null    float64
 18  Tgtrzru     334 non-null    float64
 19  Recrzru     334

In [105]:
for i in range(6):
    address = "Data/MergedPlayers/" + str(i+2016) + ".csv"
    rnk[i].to_csv(address)